In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#The purpose of this book is to compare in one plot the results of varying parameters.

name100_4 = '100nm_RED_PEG_NPs_in_0_4p_agarose'
name60_4 = '60nm_YG_PEG_NPs_in_0_4p_agarose'
name100_8 = '100nm_RED_PEG_NPs_in_0_8p_agarose'
name60_8 = '60nm_YG_PEG_NPs_in_0_8p_agarose'
name100_12 = '100nm_RED_PEG_NPs_in_1_2p_agarose'
name60_12 = '60nm_YG_PEG_NPs_in_1_2p_agarose'

geo100_4 = np.genfromtxt('./geoM2xy_{}.csv'.format(name100_4), delimiter =",")
geo60_4 = np.genfromtxt('./geoM2xy_{}.csv'.format(name60_4), delimiter =",")
geo100_8 = np.genfromtxt('./geoM2xy_{}.csv'.format(name100_8), delimiter =",")
geo60_8 = np.genfromtxt('./geoM2xy_{}.csv'.format(name60_8), delimiter =",")
geo100_12 = np.genfromtxt('./geoM2xy_{}.csv'.format(name100_12), delimiter =",")
geo60_12 = np.genfromtxt('./geoM2xy_{}.csv'.format(name60_12), delimiter =",")

In [4]:
conversion = dict()

conversion[1] = (0.10, 9.91, 1)

frames = 446
frames_1 = np.linspace(0, frames, frames+1).astype(np.int64)
time = frames_1/conversion[1][1]

In [36]:
#Figure parameters

filename = 'geo_60.png'

limit1=2.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=10.1 #y range
tick2 = 2 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [6]:
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"
    return "#" + "".join([hex(i)[2:] for i in new_rgb_int])

In [7]:
# for name, hex in mpl.colors.cnames.items():
#     print(name, hex)

# g2 = color_variant('#008000', brightness_offset=0)
# b2 = color_variant('#0000FF', brightness_offset=0)

In [8]:
def clamp(val, minimum=0, maximum=255):
    if val < minimum:
        return minimum
    if val > maximum:
        return maximum
    return val

def colorscale(hexstr, scalefactor):
    """
    Scales a hex string by ``scalefactor``. Returns scaled hex string.

    To darken the color, use a float value between 0 and 1.
    To brighten the color, use a float value greater than 1.

    >>> colorscale("#DF3C3C", .5)
    #6F1E1E
    >>> colorscale("#52D24F", 1.6)
    #83FF7E
    >>> colorscale("#4F75D2", 1)
    #4F75D2
    """

    hexstr = hexstr.strip('#')

    if scalefactor < 0 or len(hexstr) != 6:
        return hexstr

    r, g, b = int(hexstr[:2], 16), int(hexstr[2:4], 16), int(hexstr[4:], 16)

    r = round(clamp(r * scalefactor))
    g = round(clamp(g * scalefactor))
    b = round(clamp(b * scalefactor))

    return "#%02x%02x%02x" % (r, g, b)

In [9]:
g2 = colorscale('#008000', 1)
b2 = colorscale('#0000FF', 10)
r2 = colorscale('#FF0000', 10)
g2

'#008000'

In [15]:
time[0:-1].shape

(446,)

In [37]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:20], geo60_4[0:20], 'g--', linewidth=wi, dashes=[d1,d2], label='0.4%')
ax.plot(time[0:20], geo60_8[0:20], 'b--', linewidth=wi, dashes=[d1,d2], label='0.8%')
ax.plot(time[0:20], geo60_12[0:20], 'r--', linewidth=wi, dashes=[d1,d2], label='1.2%')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.80), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'geo_comparison_YG_Red.png'

limit1=20.1 #x range
tick1 = 4

limit2=2.1 #y range
tick2 =0.5

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='100nm Red')
ax.plot(time1[0:], geoM2xy_100nm_YG_NPs_in_0_9_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm YG')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical 100nm', color='purple')
# ax.plot(time1[1:], theo_MSDa[1:], '-', linewidth=10, label='theoretical 60nm', color='purple')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.65), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'geo_comparison_PEG.png'

limit1=20.1 #x range
tick1 = 4

limit2=2.1 #y range
tick2 =0.5

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS')
ax.plot(time2[0:], geoM2xy_100nm_PEG_NPs_in_0_9_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PS-PEG')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical 100nm PS', color='blue')
ax.plot(time1[1:], theo_MSDb[1:], '-', linewidth=10, label='theoretical 100nm PS-PEG', color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'geo_comparison_viscous.png'

limit1=10.1 #x range
tick1 = 2

limit2=4.1 #y range
tick2 =1

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='in 90% glycerol')
ax.plot(time4[0:], geoM2xy_100nm_RED_NPs_in_0_75_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='in 75% glycerol')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical in 90% glycerol', color='blue')
ax.plot(time4[1:], theo_MSDc[1:], '-', linewidth=10, label='theoretical in 75% glycerol', color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
geoM2xy_100nm_PS_PEG_in_0_4_agarose.shape

In [ ]:
geoM2xy_100nm_PS_PEG_in_0_4_agarose

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_60x.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_60x, 'bo', ms=mi, label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'go', ms=mi, label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_625ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, FM2xy_camera_625ms_40x, 'bo', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_1000ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_camera.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_40x, 'go', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_confocal.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'bo', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time2, line[2], linewidth=3, color='purple')
# ax.plot(time2, line[3], linewidth=3, color='purple')
# ax.plot(time1, line1[2], linewidth=3, color='purple')
# ax.plot(time1, line1[3], linewidth=3, color='purple')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_geo_ar_comparison.png'

# limit1=20.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.0051 #y range
# tick2 = round(limit2, 3)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=3 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2[0:19], geoM2xy_camera_1000ms_60x[0:-1], 'b--', linewidth=wi, dashes=[d1,d2], label='Geometric')
# ax.plot(time2[0:19], arM2xy_camera_1000ms_60x[0:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Arithmetic')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'r--', linewidth=wi, dashes=[d1,d2], label='Frame-by-frame')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()